<img style="float:right" src="https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true" alt="Insper">

# Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)
## Data Science Deploy

<h3 style="color:brown">VSP Deploy: Local e em nuvem (VMs)</h3>

WEB API com Flask: Como publicar seus modelos como serviço WEB

Local e em nuvem VPS (Virtual Private Server) Linux

A documentação online do Flask e Requests são muito boas!

<img src="https://flask.palletsprojects.com/en/2.0.x/_images/flask-logo.png" width="300" style="float:left"> 
<img src="https://requests.readthedocs.io/en/latest/_static/requests-sidebar.png" width="200" style="float:right">

Vamos publicar WEB Services localmente e<br>
Vamos publicar os mesmos WEB Services em nuvem em servidores Linux (VPS - Virtual Private Servers)

Um serviço da web (WEB Service) é: um serviço oferecido por um dispositivo eletrônico a outro dispositivo eletrônico, de máquina para máquina, comunicando-se entre si através da Internet usando protocolo TCP/IP e HTTP. Ou fornecido por um servidor, ouvindo solicitações em uma determinada porta em uma rede, servindo como resposta dados em formatos diversos (json, xml, csv, html, txt, xlsx, etc...)

Podemos criar nosso WEB Services (WEB API) em Python com o Flask, e este serviço pode ser disponibilizado em uma rede interna ou na Internet. E este recurso torna nosso código e modelos em Python acessíveis por qualquer outra linguagem de programação tal como Java, Go, C#, Ruby, PHP, entre outras. Os web services podem também ser consumidos por aplicativos WEB ou mobile (Android ou iOS).

Processo básico:

1. Criar seu modelo
2. Colocar seu modelo em uma biblioteca (arquivo .py)
3. Criar a WEB API com Flask para tornar seu modelo acessível
4. Fazer o teste local (locahost)
5. Subir o codigo do projeto no GitHub
6. Utilizar GitHub Actions para publicar o código em uma VPS em núvem
7. Testar o acesso do servico pela Internet

Documentação:

- Flask: https://flask.palletsprojects.com/en/2.0.x/
- Requests: https://docs.python-requests.org/en/latest/

Dicas de Linux:

https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Dica-019.ipynb

Com **!** no início do código em uma célula (magic) podemos executar os comandos do SO (Linux no caso)

In [11]:
!date

sáb 03 set 2022 10:30:51 -03


Podemos consumir com **curl** ou requests uma WEB API (WEB Service) que fornece dados GEO informado o nome do Insper apenas...

In [13]:
!curl "https://nominatim.openstreetmap.org/search.php?q=IBM+Brazil&format=jsonv2"

[{"place_id":111862735,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":27030323,"boundingbox":["-23.5802994","-23.5796875","-46.6495605","-46.6489428"],"lat":"-23.5799788","lon":"-46.64926899968789","display_name":"IBM, 1157, Rua Tutóia, Moema, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04007-900, Brasil","place_rank":30,"category":"building","type":"yes","importance":0.11010000000000002},{"place_id":63099672,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"node","osm_id":5780143853,"boundingbox":["-30.0462046","-30.0461046","-51.2306662","-51.2305662"],"lat":"-30.0461546","lon":"-51.2306162","display_name":"IBM, Rua Manoelito de Ornellas, Praia de Belas, Porto Alegre, Região Geográfica Imediata de Porto Alegre, Região Metropolitana de Porto Alegre, Região Geográfica Intermedi

In [14]:
import requests

URL="https://nominatim.openstreetmap.org/search.php?q=Livraria+Cultura+Sao+Paulo&format=jsonv2"

requests.get(URL).json()

#json - dict (Dicionário do Python)

[{'place_id': 80793077,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 7973675689,
  'boundingbox': ['-21.2266416', '-21.2265416', '-47.8345557', '-47.8344557'],
  'lat': '-21.2265916',
  'lon': '-47.8345057',
  'display_name': 'Livraria Cultura, Avenida Doutor Albert Sabin, Vila do Golf, Bonfim Paulista, Ribeirão Preto, Região Imediata de Ribeirão Preto, Região Metropolitana de Ribeirão Preto, Região Geográfica Intermediária de Ribeirão Preto, São Paulo, Região Sudeste, 14027-050, Brasil',
  'place_rank': 30,
  'category': 'shop',
  'type': 'books',
  'importance': 0.30010000000000003}]

Este exemplo de WEB Service, retorna um JSON (dict) com informações de clima dado a GEO localização, latitude o longitude do Insper:

In [15]:
import requests

KEY="e5cdb1ac5452815cc556ad568df2c2e8"
LAT=-23.5987249
LON=-46.67643049021903

URL=f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={KEY}"

requests.get(URL).json()

{'coord': {'lon': -46.6764, 'lat': -23.5987},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 291.27,
  'feels_like': 291.04,
  'temp_min': 290.46,
  'temp_max': 294.28,
  'pressure': 1023,
  'humidity': 73},
 'visibility': 10000,
 'wind': {'speed': 4.12, 'deg': 200},
 'clouds': {'all': 75},
 'dt': 1662212221,
 'sys': {'type': 1,
  'id': 8446,
  'country': 'BR',
  'sunrise': 1662196552,
  'sunset': 1662238604},
 'timezone': -10800,
 'id': 3461570,
 'name': 'Ibirapuera',
 'cod': 200}

WEB API para busca de CEP: https://viacep.com.br/

In [16]:
import requests

requests.get("https://viacep.com.br/ws/01001000/json/").json()

{'cep': '01001-000',
 'logradouro': 'Praça da Sé',
 'complemento': 'lado ímpar',
 'bairro': 'Sé',
 'localidade': 'São Paulo',
 'uf': 'SP',
 'ibge': '3550308',
 'gia': '1004',
 'ddd': '11',
 'siafi': '7107'}

In [17]:
!curl viacep.com.br/ws/03819250/json/

{
  "cep": "03819-250",
  "logradouro": "Rua Água Nova",
  "complemento": "",
  "bairro": "Parque Císper",
  "localidade": "São Paulo",
  "uf": "SP",
  "ibge": "3550308",
  "gia": "1004",
  "ddd": "11",
  "siafi": "7107"
}

In [18]:
!curl viacep.com.br/ws/03819250/xml/

<?xml version="1.0" encoding="UTF-8"?>
<xmlcep>
  <cep>03819-250</cep>
  <logradouro>Rua Água Nova</logradouro>
  <complemento></complemento>
  <bairro>Parque Císper</bairro>
  <localidade>São Paulo</localidade>
  <uf>SP</uf>
  <ibge>3550308</ibge>
  <gia>1004</gia>
  <ddd>11</ddd>
  <siafi>7107</siafi>
</xmlcep>

Note como fica a URL que faz a chamada da WEB API, isso funciona no browser também... pois estamos usando o protocolo HTTP/S, para criar WEB Services (WEB API) utiliza-se os mesmos recursos da tecnologia WEB... dado sua simplicidade e accessibilidade...

- Uma visão geral do HTTP: https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Overview

<img src="https://igluonline.com/wp-content/uploads/2018/12/como-funciona-http.jpg.jpg" width="600px">

In [10]:
URL

'https://api.openweathermap.org/data/2.5/weather?lat=-23.5987249&lon=-46.67643049021903&appid=e5cdb1ac5452815cc556ad568df2c2e8'

> Agora vamos criar nosso próprios serviços WEB (WEB Servie ou WEB API) com Flask

In [ ]:
!conda install flask

In [4]:
!pip install flask

In [ ]:
!conda install requests

In [5]:
!pip install requests

Agora vamos trabalhar fora do ambiente Jupyter, vamos trabalhar com python em arquivos texto .py, ambiente script na linha de comando. Isso funciona no Linux, Mac ou Windows.

Criar uma pasta de tralbalho (projeto), e criar o arquivo **insperds.py** que é nossa biblioteca de funções

```Python
import requests

def ddgquery(query):
    URL = "https://api.duckduckgo.com/?format=json&pretty=0&q=" + query
    response = requests.get(URL)
    ddg = response.json()
    info = ddg["Abstract"]
    if len(info) > 1:
        return info
    else:
    	return "Ops! No data found!" 

def bitcoins():
    URL="https://www.mercadobitcoin.net/api/BTC/trades/"
    response = requests.get(URL)
    return {"btc":response.json()}

def ethereum():
    URL="https://www.mercadobitcoin.net/api/ETH/trades/"
    response = requests.get(URL)
    return {"eth":response.json()}

def weather(latitude, longitude):
    KEY = "ed38d2abac6e6aded3cf1ed68fddb3c6"
    URL = f'https://api.openweathermap.org/data/2.5/weather?lat=' + str(latitude) + '&lon=' + str(longitude) + '&appid='+ KEY
    return requests.get(URL).json()
```

Agora vamos criar o script que executa as WEB APIs em Flask **core.py**

```python
# A simple Flask Hello World WEB API to get started with...

from flask import Flask
from flask import request
from datetime import datetime
import insperds

app = Flask(__name__)

@app.route('/')
def hello_world():
    now = datetime.now()
    return 'Hello from Flask, by Insper DS! - ' + str(now)

@app.route('/core')
def core():
    now = datetime.now()
    return 'Vai Corinthians!! - ' + str(now) 

@app.route('/add/<a>/<b>')
def add(a, b):
    return str(float(a) + float(b))

@app.route('/area')
def myarea():
  altura = request.args.get('altura', default = 0, type = float)
  largura = request.args.get('largura', default = 0, type = float)
  comprimento = request.args.get('comprimento', default = -1, type = float)
  
  if (comprimento < 0):
        return str(altura*largura)
  else:
        return str(altura*largura*comprimento)

@app.route('/query/<text>')
def query(text):
    return insperds.ddgquery(text)
    
@app.route('/bitcoins')
def btc():
    return insperds.bitcoins()
    
@app.route('/ethereum')
def ether():
    return insperds.ethereum()
    
@app.route('/weather/<lat>/<lon>')
def weather(lat, lon):
    return insperds.weather(lat, lon)
            
```

Precisamos instalar o Flask no sistema operacional (SO - Linux e Mac):
    
```bash
sudo apt install flask
```

Podemos executar os comandos abaixo (Linux e Mac) ou criar um script **run.sh**


```bash
export FLASK_APP=core
flask run --host=0.0.0.0
```

A porta padrão do Flask é a 5000, lembre-se de liberar esta porta no firewall...<br>
O comando host com o IP 0.0.0.0 significa que esta aplicação atende a chamada (inbound) proveniente de qualquer endereço IP<br>
O comando export cria uma variável de ambiente no SO (chave valor)

> Outra forma de executar um progra Flask na linha de comando, por exemplo, app.py:<br>
> flask --host=0.0.0.0 --port=5000 run app

A janela do Flask em linha de comando fica aberta e podemos ver as requisições no log.

```bash
fito@acme-lab:/work/Insper/workshop3/flask-local$ sh run.sh
 * Serving Flask app "core"
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [1/Feb/2022 02:54:12] "GET /core HTTP/1.1" 200 -
127.0.0.1 - - [1/Feb/2022 02:54:25] "GET /add/10/333 HTTP/1.1" 200 -
127.0.0.1 - - [1/Feb/2022 02:54:38] "GET /query/brasil HTTP/1.1" 200 -


```

É possível também no Linux (Bash) deixar o flask rodando em background (segundo plano - serviço)

```bash
nohup flask --host=0.0.0.0 --port=5000 run app &
```

Desta forma mesmo fechando a conexão ssh o flask continua em execução, a saída do log é acumulada no arquivo "nohup.out". Veja mais em: https://linux.die.net/man/1/nohup

Podemos testar o serviço com o utilitário CLI **curl** ou no próprio notebook com o **requests**, com o browser é possível para o verbo HTTP GET

In [1]:
!curl http://localhost:5000/core

Vai Corinthians!! - 2022-02-25 19:53:08.021404

In [2]:
!curl http://localhost:5000/add/10/333

343.0

In [3]:
!curl http://localhost:5000/query/brasil

Brazil, officially the Federative Republic of Brazil, is the largest country in both South America and Latin America. At 8.5 million square kilometers and with over 211 million people, Brazil is the world's fifth-largest country by area and the sixth most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the largest country to have Portuguese as an official language and the only one in the Americas; it is also one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world; as well as the most populous Roman Catholic-majority country. Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers. It borders all other countries in South America except Ecuador and Chile and covers 47.3% of the continent's land area.

Agora o texto usando o requests no Jupyter notebook:

In [6]:
import requests

r=requests.get("http://localhost:5000/query/Microsoft")
r.text

"Microsoft Corporation is an American multinational technology corporation which produces computer software, consumer electronics, personal computers, and related services. Its best-known software products are the Microsoft Windows line of operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers. Microsoft ranked No. 21 in the 2020 Fortune 500 rankings of the largest United States corporations by total revenue; it was the world's largest software maker by revenue as of 2016. It is one of the Big Five American information technology companies, alongside Alphabet, Amazon, Apple, and Meta. Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800."

O processo para colocar o serviço no ar em nosso computador local (Linux ou Mac) é igual para colocar estes serviços em uma máquina virtual Linux (servidor) em nuvem (VPS - Virtual Private Server).

O ambiente em nuvem precisamos configurar o domínio (DNS) e a camada de seguranças (Firewalls etc...), conforme cada provedor.

### Máquina Virtual (VM) Linux em nuvem

Virtual Private Server (VPS)

Linux Ubuntu - https://ubuntu.com/

Testes no Oracle Cloud:

```bash
root@web-app-server-01:~# hostname
web-app-server-01
root@web-app-server-01:~# uptime
 01:37:04 up 10 days,  6:37,  1 user,  load average: 0.02, 0.01, 0.00
root@web-app-server-01:~# uname -a
Linux web-app-server-01 5.11.0-1027-oracle #30~20.04.1-Ubuntu SMP Thu Jan 13 09:43:40 UTC 2022 aarch64 aarch64 aarch64 GNU/Linux
root@web-app-server-01:~# python3 --version
Python 3.8.10
```

Testes no Microsoft Azure:

```bash
fitoplancton@app-07:~$ hostname
app-07
fitoplancton@app-07:~$ uptime
 02:36:50 up 370 days, 20:49,  1 user,  load average: 0.00, 0.03, 0.02
fitoplancton@app-07:~$ uname -a
Linux app-07 4.18.0-1013-azure #13~18.04.1-Ubuntu SMP Thu Feb 28 23:48:47 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
fitoplancton@app-07:~$ python3 --version
Python 3.6.9
```

Podemos conectar so servidor Linux (VM) em nuvem pela linha de comando usando o **ssh** no Linux e Mac:


Oracle...<br>
**ssh -i ssh-key-2022-02-13.key ubuntu@w1.code.eng.br**

Microsoft...<br>
**ssh fitoplancton@lab.code.eng.br**

ou<br>
**ssh maria@253.34.223.3**

Para client Windows utilizamos o PuTTY - https://www.putty.org/

Note que as versões do Python disponíveis são diferentes, no seu ambiente local e entre os servidores!!! E não foi instalado com o Anaconda.

Vamos ver na máquina local:

In [16]:
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [19]:
!python --version

Python 3.8.5


In [20]:
!conda --version

conda 4.13.0


**LAB!** Vamos para o ambiente em um servidor Linux em nuvem - VPS!!!

 > Será realizada demonstração de alguns tópicos do workshop/aula. Os serviços em nuvem requerem registro e cadastro com catão de crédito e em alguns casos CNPJ também. E vamos experimentar serviços de diferentes provedores... há riscos financeiros ao utilizar ambientes de *cloud computing* mesmo que para estudo e testes. Fique atento para não deixar recursos ligados após o uso, pois a cobrança é feita por hora!

### Microsoft Azure - VM Linux

Podemos obter acesso a recursos gratuitos na núvem da Microsoft, usando o e-mail do Insper:

https://azure.microsoft.com/pt-br/free/students/

Conectando via SSH (Linha de comando)

```bash
fito@acme-lab:/work/Insper/Insper-Data-Science-Deploy$ ssh fitoplancton@lab.code.eng.br
fitoplancton@lab.code.eng.br's password: 
Welcome to Ubuntu 18.04.2 LTS (GNU/Linux 4.18.0-1013-azure x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Fri Feb 25 12:26:16 UTC 2022

  System load:  0.1                Processes:           118
  Usage of /:   31.0% of 28.90GB   Users logged in:     0
  Memory usage: 37%                IP address for eth0: 10.0.0.4
  Swap usage:   5%

 * Ubuntu's Kubernetes 1.14 distributions can bypass Docker and use containerd
   directly, see https://bit.ly/ubuntu-containerd or try it now with

     snap install microk8s --channel=1.14/beta --classic

  Get cloud support with Ubuntu Advantage Cloud Guest:
    http://www.ubuntu.com/business/services/cloud

 * Canonical Livepatch is available for installation.
   - Reduce system reboots and improve kernel security. Activate at:
     https://ubuntu.com/livepatch

131 packages can be updated.
3 updates are security updates.


*** System restart required ***
Last login: Thu Feb 24 02:21:13 2022 from 201.83.118.214
fitoplancton@app-07:~$ 

```

Acesso via IP público ou pelo domínio... liberar a porta no Firewall...

In [7]:
!curl http://lab.code.eng.br:5000/

Hello from Flask, by Insper DS! - 2022-02-25 23:10:29.274893

In [8]:
!curl http://lab.code.eng.br:5000/query/Insper

The Insper Learning Institution is a private college, that offers Higher Education in fields of Business, Economics, Engineering, Law and Computer Science in Latin America, located in São Paulo, Brazil.

Console do Flask no servidor Linux na Macrosoft Azure (Note que está sem chave SSL, protocolo HTTPS)

<img src="img/flask-azure.png">

Este serviço pode ser acessado de algumas formas diferentes, pelo número IP, pelo domínio (DNS), ou pelo domínio fornecido pelo provedor:

In [9]:
!curl http://13.77.179.189:5000/add/23/44

67.0

In [10]:
!curl http://lab.code.eng.br:5000/add/66/124

190.0

In [11]:
!curl http://eng.westus2.cloudapp.azure.com:5000/add/99.99/-244

-144.01

Lembrando que o DNS (Domain Name System), ou domínio, aponta para um endereço IP (Internet Protocol).

Um número IP é um endereço exclusivo que identifica um dispositivo ou servidor (físico ou virtual) na Internet ou em uma rede local.

- [DNS](https://en.wikipedia.org/wiki/Domain_Name_System)
- [IP](https://www.kaspersky.com/resource-center/definitions/what-is-an-ip-address)

> Há nos ambientes em nuvem como da Microsoft Azure, máquinas virtuais prontas (templates) para ciências dos dados, um tipo de marketplace, que já vem com o Anaconda, Python, Flask, etc... já pré instalado

### Rede e Segurança

Para o domínio de lab (válido na Internet) **code.eng.br**, configuramos os IPs dos serviços em núvem no gerenciador de DNS (Domínios), apontado os IPs para um subdomínio:
<img src="img/dns-subdominio.png">
Esta configuração é normalmente feita no ambiente WEB do provedor de DNS.

Um domínio custa por volta de 30,00 a 40,00 reais ao ano.

Verificando no Bash (Linux) para onde o domínio aponta, para qual número IP:

```bash
root@acme-lab:/work/Insper$ nslookup w1.code.eng.br
Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
Name:	w1.code.eng.br
Address: 129.153.157.106

```

Podemos também verificar a propriedade do domínio:

```bash
root@acme-lab:/work/Insper$ whois w1.code.eng.br

% Copyright (c) Nic.br
%  The use of the data below is only permitted as described in
%  full by the terms of use at https://registro.br/termo/en.html ,
%  being prohibited its distribution, commercialization or
%  reproduction, in particular, to use it for advertising or
%  any similar purpose.
%  2022-02-25T02:21:57-03:00 - IP: 2804:14c:c6:878d:a64b:38e2:46b0:15de

domain:      code.eng.br
owner:       Insper
ownerid:     333.777.999-11
country:     BR
owner-c:     ATLME159
tech-c:      ATLME159
nserver:     ns1.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
nserver:     ns2.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
nserver:     ns3.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
saci:        yes
created:     20200629 #21281503
changed:     20210610
expires:     20220629
status:      published
provider:    LOCAWEB (2)

nic-hdl-br:  ATLME159
person:      Insper
e-mail:      danielscarvalho@gmail.com
country:     BR
created:     20200629
changed:     20200629
provider:    LOCAWEB (2)

% Security and mail abuse issues should also be addressed to
% cert.br, http://www.cert.br/ , respectivelly to cert@cert.br
% and mail-abuse@cert.br
%
% whois.registro.br accepts only direct match queries. Types
% of queries are: domain (.br), registrant (tax ID), ticket,
% provider, CIDR block, IP and ASN.
```

O [Firewall](https://pt.wikipedia.org/wiki/Firewall) é um recurso de segurança da informação, que protege uma rede de outra, neste caso protege o servidor virtual no provedor do tráfego de dados que vem da Internet.

Um Firewall é um dispositivo de segurança de rede que monitora e filtra o tráfego de rede de entrada e saída com base nas políticas de segurança previamente estabelecidas de uma organização. Em sua forma mais básica, um firewall é essencialmente a barreira que fica entre uma rede interna privada e a Internet pública.

Alguns provedores tem um nível de firewall (1) configurado via WEB em sua central de controle.<br>

Outros provedores tem dois nívels de seguranças (2) com o software de firewall adicional na máquina virtual (VM) o [iptables](https://www.netfilter.org/), como no Oracle Cloud por exemplo.
<img src="img/firewall-diagram.png">
É necessário nos serviços em núvem, liberar as portas para acesso aos serviços, note que neste caso as portas 22, 443, 80, 8080 e 5000 estão liberadas para entradas (inbound), mas todas as outras estão bloqueadas por seguranças:
<img src="img/regras-firewall.png">

Endereços IP (número IP) são atribuidos ao servidores (físico ou virtual). Os IPs são atribuídos as placas de rede, diversos IPs podem ser atribuídos a mesma placa de rede. E o servidor pode ter diversas placas de rede também.

Temos 2 versões de protocolo de rede TCP/IP em uso IPv4 e IPv6.

Note que temos IPv4, 4 conjunto de 3 números separador por ".", exemplo:

    Localhost: 127.0.0.1
    IP público: 108.139.182.81

IPv6: 8 conjuntos de 4 números separados por ":", exemplo: 

    Localhost: 0000:0000:0000:0000:0000:0000:0000:0001 ou ::1
    IP público: 2600:9000:21ed:bc00:1:5a19:8b40:93a1

Além disto temos as portas, cada programa utiliza uma porta para se comunicar (listen), das portas 0 até 1024 são exclusivas do root (super usuário). São atribuídos serviços padronizados para as portas, conforme segue:

<img src="https://dirwalk.com/images/common_ports_cheatsheet.jpg" width="600px">

O IP é o "endereço" do servidor, e a porta seria qual "apartamento" deve receber os pacotes! 😜 

O DNS é o "apelido" do endereço, para não termos que decorar os números. O nome (DNS) aponta para um número IP público válido na Internet.

Há outros aspectos de segurança que são importantes como chave de criptografia (SSL, TTL), ACL (Access Control List), etc...

Quando a chamada HTTP é feita para o serviço do Flask que está no servidor, note que o mair tempo se leva na espera do processamento do servidor para realizar computação:

<img src="img/network-timing.png">

Este nível de detalhe pode ser visto no console do browser, ferramenta utlizada por developers. Techas **Ctrl + Shift + i**, pasta Network.

Podemos usar o comando **nohup** do Linux para sair do ambiente e deixar o Flask no are como serviço do Linux.

- https://linux.die.net/man/1/nohup

E também podemos ter várias instâncias do Flask em execução simultaneamente no mesmo servidor Linux (VM) em segundo plano, em portas diferentes (que devem ser liberadas no Firewall, é claro):

```bash
nohup flask --host=0.0.0.0 --port=5001 run app1 &
nohup flask --host=0.0.0.0 --port=5002 run app2 &
nohup flask --host=0.0.0.0 --port=5003 run app3 &
```

Use o comando **htop** para ver os processo em execução em segundo plano. Ou o comando **ps -ef | grep flask**<br>
Para derrubar um processo em segundo plano use o comando **kill -9 zzzz**, sendo zzzz o número do processo.

Todos os ambientes do provedores serviços em nuvem, fornecem máquinas virtuais (VM ou VPS).

São serviços mais flexíveis porem você é responsável por instalar, manter, atualizar, configura, etc... é como ter o seu próprio computador exposto na Internet 24x7x365.

O preço destes serviço variam conforme a capacidade de processamento das VMs, processador, disco, memória e banda Internet. E se os recursos são compartilhados ou dedicados. 

O ambiente do Google Cloud para VMs é chamado *Google Compute Engine*:

<img src="https://cloud.google.com/languages/images/compute-single-instance.svg">

https://cloud.google.com/python/docs/getting-started/getting-started-on-compute-engine

### Extra

Dica senha segura:

Ao usar serviços na Internet, use senhas seguras geradas automaticamente e também use gerenciador de senhas.


A sua senha vai ser quebrada!!<br>
ACREDITE NISSO!!!

- https://password-gen.com/ >> Para não quebrar a sua senha!
- https://www.lastpass.com/pt/features/password-generator >> Gerador de senha segura
- https://keepass.info/ >> Para você lembrar a senha

Exemplos de senha segura:

<pre>
*dk*7OBx7l3p!z5L
^cketGx#NS52^o7^
3j#98v75p2HyR!1o
</pre>

Os computadores pessoais PC Linux, Mac ou Windows tem firewall local, ou seja, se você subir um servoiço com Flask e passar o número IP do seu computador para seus colegas na mesma rede, eles não vão ter acesso. Você vai ter que liberar a porta 5000 (padrão do Flask) no seu firewall local.

Para saber seu IP da rede local no Linux use o comando **ifconfig**, no windows **ipconfig**.

In [7]:
!ifconfig

docker0: flags=4099<UP,BROADCAST,MULTICAST>  mtu 1500
        inet 172.17.0.1  netmask 255.255.0.0  broadcast 172.17.255.255
        ether 02:42:b3:73:80:60  txqueuelen 0  (Ethernet)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

enp3s0: flags=4099<UP,BROADCAST,MULTICAST>  mtu 1500
        ether d0:94:66:a1:bd:2a  txqueuelen 1000  (Ethernet)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 18115  bytes 20664849 (20.6 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 1

Note que há ranges de IPs reservados para uso em redes locais que não podem ser usados na Internet para domínio e sites públicos:

<pre>
192.x.x.x
10.x.x.x
100.x.x.x
172.x.x.x
</pre>

Mais detalhes em: https://en.wikipedia.org/wiki/Reserved_IP_addresses

Para saber seu IP válido na Internet, que é diferente do seu IP da rede local que é protegido... use o link: https://whatismyipaddress.com/ Isso é útil para realizar testes e configuração de firewall. (CUIDADO - Não compartilhe esta informação)


### Dicas:

Para pegar seu IP local com Python:

In [18]:
import socket

def extract_ip():
    st = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:       
        st.connect(('10.255.255.255', 1))
        IP = st.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        st.close()
    return IP

extract_ip()

'10.99.90.141'

A sua rede local, tanto em casa, no trabalho ou na faculdad (Insper), ao conectar seu computador na rede via cabo ou wi-fi, o serviço de DHCP empresa um IP local para você.

Para pegar via Python o seu IP público na Interne

(CUIDADO! PERIDGO! NÃO DIVULGUE ESTA INFORMAÇÃO)

In [21]:
from requests import get

ip = get('https://api.ipify.org').text

ip

'200.173.171.24'

Quando tiver problemas de rede para acessar WEB APIs (inbound) ou publicar WEB APIs (outbound) na rede da sua empresa, fale com o pessoal de suporte, eles tem profundo conhecimento sobre redes e podem ajudar a resolver o problema e manter a segurança.

Os desktops Linux, Windows e Mac normalmente são protegidos por firewall local. Se tornar um serviço ativo no seu computador, provavelmente ele não fica acessível pelo seu IP na rede local. Mesmo sendo acessível para você no próprio **localhost*** (127.0.0.1).

Note que o jupyter notebook também é uma aplicação WEB que roda local no seu computador, ele abre uma janela de console e mostra como acessar a interface WEB, algo do tipo:<br>
```bash
    To access the notebook, open this file in a browser:
        file:///home/dan/.local/share/jupyter/runtime/nbserver-40851-open.html
    Or copy and paste one of these URLs:
        http://localhost:8888/?token=ddad4c35ad80922dab01e17dc7c7fd1e4a89ec718dbc9b87
     or http://127.0.0.1:8888/?token=ddad4c35ad80922dab01e17dc7c7fd1e4a89ec718dbc9b87

```

### Atividade (LAB):


Seguir o tutorial da Microsoft com Flask para colocar um serviço em Python com Flask disponível na Internet:

https://docs.microsoft.com/en-us/samples/azure-samples/flask-app-on-azure-functions/azure-functions-python-create-flask-app/

Estudar a documentação e tutorial oficial no site do Flask:

https://flask.palletsprojects.com/

### Para aprender sobre redes:

Leia um livro sobre redes de computadores, Internet, TCP/IP, HTTP, etc...

- [Computer Networks (5th Edition): Tanenbaum, Andrew](https://www.amazon.com/Computer-Networks-5th-Andrew-Tanenbaum/dp/0132126958)
- [15 BEST Computer Networking Books (2022 Update)](https://www.guru99.com/best-computer-networks-books.html)